In [1]:
!pip install PyMuPDF jsonlines


   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
    ----------------

In [2]:
import os
import fitz  # PyMuPDF para leer PDFs
import jsonlines

In [3]:
pdf_folder = "contratos_pdf/"
categories = {
    "regular": "✅ Regular",
    "irregular": "❌ Irregular"
}


In [ ]:
dataset = []

def extract_text_from_pdf(pdf_path):
    """ Extrae el texto de un archivo PDF """
    with fitz.open(pdf_path) as doc:
        text = "\n".join([page.get_text() for page in doc])
    return text.strip()


In [ ]:
# 📂 Recorrer cada categoría (regular/irregular)
for category, label in categories.items():
    folder_path = os.path.join(pdf_folder, category)
    
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith(".pdf"):
                pdf_path = os.path.join(folder_path, filename)
                text = extract_text_from_pdf(pdf_path)
                
                # 📌 Crear entrada para el dataset
                sample = {
                    "prompt": f"Analiza este contrato y determina si cumple con la ley dominicana:\n\n{text}",
                    "response": label
                }
                
                dataset.append(sample)

# 📝 Guardar el dataset en formato JSONL
output_jsonl = "contratos_dataset.jsonl"
with jsonlines.open(output_jsonl, "w") as writer:
    writer.write_all(dataset)

print(f"\n✅ Dataset generado y guardado en {output_jsonl}")

In [2]:
!pip install chardet


In [ ]:
input_file = "CONTRATO DE HIPOTECA CONVENCIONAL EN PRIMER RANGO.txt"
output_file = "CONTRATO DE HIPOTECA CONVENCIONAL EN PRIMER RANGO_UTF8.txt"

try:
    with open(input_file, "r", encoding=encoding_detectado, errors="ignore") as file:
        content = file.read()

    with open(output_file, "w", encoding="utf-8") as file:
        file.write(content)

    print("✅ Archivo convertido a UTF-8 correctamente.")
except PermissionError:
    print("❌ Error: No se tienen permisos para escribir en el archivo.")
except Exception as e:
    print(f"❌ Error: {e}")

❌ Error: name 'encoding_detectado' is not defined


In [24]:
import chardet

input_file = "CONTRATO DE ALC.txt"

# Detectar codificación
with open(input_file, "rb") as file:
    result = chardet.detect(file.read())

print(f"📌 La codificación detectada es: {result['encoding']}")


📌 La codificación detectada es: ISO-8859-1


In [26]:
# Convertir archivo a UTF-8 si es necesario
encoding_detectado = result["encoding"]

if encoding_detectado.lower() != "utf-8":
    with open(input_file, "r", encoding=encoding_detectado, errors="ignore") as file:
        content = file.read()

    with open(input_file, "w", encoding="utf-8") as file:
        file.write(content)

print("✅ Archivo convertido a UTF-8 correctamente.")


✅ Archivo convertido a UTF-8 correctamente.


In [44]:
import re
import jsonlines

# 📂 Ruta del archivo de contratos
input_file = "CONTRATO DE VENTA DE VEHÍCULO DE MOTOR_UTF8.txt"  # Guardado en texto
output_jsonl = "contratos_dataset_VM.jsonl"

# 📌 Expresiones regulares para detectar títulos de contratos
regex_regular = re.compile(r"Generado\s+regular\s+\d+", re.IGNORECASE)
regex_irregular = re.compile(r"Generado\s+irregular\s+\d+", re.IGNORECASE)

# 📌 Variables de control
dataset = []
current_contract = []
current_label = None

# 📂 Leer el archivo línea por línea
with open(input_file, "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()

        # Detectar si es un contrato regular
        if regex_regular.match(line):
            if current_contract:
                dataset.append({
                    "prompt": f"Analiza este contrato y determina si cumple con la ley dominicana:\n\n{' '.join(current_contract)}",
                    "response": "✅ Regular"
                })
                current_contract = []

            current_label = "✅ Regular"

        # Detectar si es un contrato irregular
        elif regex_irregular.match(line):
            if current_contract:
                dataset.append({
                    "prompt": f"Analiza este contrato y determina si cumple con la ley dominicana:\n\n{' '.join(current_contract)}",
                    "response": "❌ Irregular"
                })
                current_contract = []

            current_label = "❌ Irregular"

        # Agregar contenido del contrato
        elif line:
            current_contract.append(line)

# Guardar el último contrato después del bucle
if current_contract and current_label:
    dataset.append({
        "prompt": f"Analiza este contrato y determina si cumple con la ley dominicana:\n\n{' '.join(current_contract)}",
        "response": current_label
    })

# 📝 Guardar el dataset en JSONL
with jsonlines.open(output_jsonl, "w") as writer:
    writer.write_all(dataset)

print(f"\n✅ Dataset generado y guardado en {output_jsonl}")



✅ Dataset generado y guardado en contratos_dataset_VM.jsonl
